In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_rows', 20)
pd.set_option('display.max_columns', 500)

In [2]:
import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

In [3]:
from utils import UtilsKy
from analyzer import AnalyzerPrediction

In [4]:
# for autoreload modules
%load_ext autoreload
%autoreload 2

In [5]:
# pytorch mlp for binary classification
from numpy import vstack
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset 
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torch import Tensor
from torch.nn import Linear
from torch.nn import ReLU
from torch.nn import Sigmoid
from torch.nn import Module
from torch.optim import SGD
from torch.nn import BCELoss
from torch.nn.init import kaiming_uniform_
from torch.nn.init import xavier_uniform_
import joblib

In [6]:
db_teach = pd.read_csv( UtilsKy.DB_TEACH_KYW3, dtype=str, encoding='cp1251')

In [7]:
db_test = pd.read_csv(UtilsKy.DB_TEST_KYW3, dtype=str, encoding='cp1251')

In [8]:
white = pd.read_csv(UtilsKy.WHITE_KYW3 , dtype=str)

In [9]:
COL_FACTORS = ['amount', 'bank_currency', 'bin', 'count_months_to_end_card', 'day_of_week', 'is_city_resolved', 'hour',
                                                             'is_gender_undefined', 'latitude', 'longitude', 'phone_2_norm']
COL_FACTORS = sorted(COL_FACTORS)

In [10]:
from helper import DataHelper
datahelper = DataHelper(db_teach, db_test, COL_FACTORS)
datahelper.create_train_test()
datahelper.show_columns_with_na()
mean_values = datahelper.get_mean_value()
replaced_values = { col: mean_values[col] for col in ('latitude', 'longitude')}
replaced_values['default'] =  -999
datahelper.replaced_na_values(replaced_values) 
scaler_params = datahelper.get_scaler_params()
datahelper.minMaxScaler_own()
datahelper.add_status_in_train_test()
train , test = datahelper.get_train_test()

train na columns : Index(['latitude', 'longitude'], dtype='object')
test na columns : Index(['latitude', 'longitude'], dtype='object')
-92.53325861542274
-999
-999
-999
36.90237577890762
-999
-999
-999
-999
-999
-999


In [11]:
class CSVDataset(Dataset):
    # load the dataset
    def __init__(self, df): # path
        # store the inputs and outputs
        self.X = df.values[:, :-1]
        self.y = df.values[:, -1]
        # ensure input data is floats
        self.X = self.X.astype('float32')
        # label encode target and ensure the values are floats
        self.y = LabelEncoder().fit_transform(self.y)
        self.y = self.y.astype('float32')
        self.y = self.y.reshape((len(self.y), 1))
 
    # number of rows in the dataset
    def __len__(self):
        return len(self.X)
 
    # get a row at an index
    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]
 
    # get indexes for train and test rows
    def get_splits(self, n_test=0):
        # determine sizes
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        # calculate the split
        return random_split(self, [train_size, test_size])

In [12]:
# model definition
class MLP(Module):
    # define model elements
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        # input to first hidden layer
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        # second hidden layer
        self.hidden2 = Linear(10, 6)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        # third hidden layer and output
        self.hidden3 = Linear(6, 1)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Sigmoid()
 
    # forward propagate input
    def forward(self, X):
        # input to first hidden layer
        X = self.hidden1(X)
        X = self.act1(X)
         # second hidden layer
        X = self.hidden2(X)
        X = self.act2(X)
        # third hidden layer and output
        X = self.hidden3(X)
        X = self.act3(X)
        return X

In [13]:
# prepare the dataset
def prepare_data(df):
    # load the dataset
    dataset = CSVDataset(df)
    # calculate split
    train, test = dataset.get_splits()
    # prepare data loaders
    train_dl = DataLoader(train, batch_size=32, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

In [14]:
def train_model(train_dl, model, optimizer=None, n_epoch=100, lr=0.01, momentum=0.9):
    criterion = BCELoss()
    if optimizer == None:
        optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)   
    for epoch in range(n_epoch):
        for i, (inputs, targets) in enumerate(train_dl):
            optimizer.zero_grad()
            yhat = model(inputs)
            loss = criterion(yhat, targets)
            loss.backward()
            optimizer.step()

In [15]:
def predict(row, model):
    # convert row to data
    row = Tensor([row])
    # make prediction
    yhat = model(row)
    # retrieve numpy array
    yhat = yhat.detach().numpy()
    return yhat

In [16]:
def get_db_test_prediction(db_test, model, numeric_cols):
    predictions = list()
    test = db_test[numeric_cols].values
    for row in test:
        predictions.append(predict(row, model)[0][0])
    return predictions         

In [17]:
train_dl, _ = prepare_data(train)

In [18]:
n_features = len(COL_FACTORS)
model = MLP(n_features)

In [19]:
model.__class__

__main__.MLP

In [20]:
analyzer_prediction =  AnalyzerPrediction(db_teach, db_test, white)

In [21]:
result_df_amount = None

In [22]:
n_epoch = 30
lr = 0.01 
momentum = 0.9

optimizer = SGD(model.parameters(), lr=lr, momentum=momentum)
train_model(train_dl, model, optimizer, n_epoch=n_epoch, lr=lr, momentum=momentum)

test_probability = get_db_test_prediction(test, model, COL_FACTORS)
db_test["probability"] = test_probability

description = '-' . join([str(elem) for elem in (n_epoch, momentum, lr)])       
result_df_amount = analyzer_prediction.get_table_prediction(description=description, result_df=result_df_amount, metric="amount")
print( 'n_epoch={}'. format(n_epoch ))

n_epoch=30


In [23]:
n = result_df_amount.shape[0]
sub_rows = list(range(n))[::2]
stat_best = result_df_amount.copy().iloc[sub_rows,:]

col_names = [col for col in stat_best.columns if col.startswith('p_') ] 
stat_best.loc[:, col_names] = stat_best.loc[:, col_names].astype(float)
stat_best = stat_best.sort_values(by="rating", ascending=False)

In [24]:
stat_best.iloc[:,:15]

,description,p_1,p_2,p_3,p_4,p_5,p_6,p_7,p_10,p_20,rating,n_white_list,n_test_in_wl,n_test_bad_in_wl,amount_test_in_wl
0,30-0.9-0.01,5.22,9.55,13.62,15.9,18.86,21.91,26.18,32.19,48.57,143.91,1019125,22992,34,1640236.53


## Save model 

In [78]:
conf_model = {'profile': model, 'algorithm_name': 'pytorch', 'factor_list': COL_FACTORS, 'replaced_values': replaced_values, 
              'scaler_params': scaler_params}

In [79]:
from datetime import date
today = date.today()
today = today.strftime("%Y-%m-%d")
today

'2021-01-25'

In [80]:
import re 
file_name =  '_' . join([str(elem) for elem in ('pytorch', 'kyw3',  description, today)]) 
file_name  = re.sub("[.]", "", file_name)

In [82]:
file_class_name =re.sub("[-]", "_", file_name)
file_class_name

'pytorch_kyw3_30_09_001_2021_01_25'

In [81]:
joblib.dump(conf_model, file_name)

['pytorch_kyw3_30-09-001_2021-01-25']

## Load model

## We need to modlel class(MLP) For load model